In [ ]:
import os
import sys
import numpy as np
import cv2
print("HELLO")
os.chdir('/home/jonfrey/PLR2')
sys.path.append('src')
sys.path.append('src/dense_fusion')

from loaders_v2 import Backend, ConfigLoader, GenericDataset
from PIL import Image
import copy
from helper import re_quat
from scipy.spatial.transform import Rotation as R

exp_cfg = ConfigLoader().from_file('/home/jonfrey/PLR2/yaml/exp/exp_ws_deepim.yml')
env_cfg = ConfigLoader().from_file('/home/jonfrey/PLR2/yaml/env/env_natrix_jonas.yml')
print(env_cfg)
generic = GenericDataset(
    cfg_d=exp_cfg['d_train'],
    cfg_env=env_cfg)

img = Image.open('/media/scratch1/jonfrey/datasets/YCB_Video_Dataset/data/0000/000001-color.png')



In [ ]:
out  = generic[0]
generic.visu = True
names = ['cloud','choose','img_masked','target','model_points','idx','add_depth','add_mask','img','cam','rot','trans','desig']

sample = {}
print(len(out[0]))
for i,o in enumerate(out[0]):    
    sample[names[i]] = o

for n in names:
    try:
        print(n, sample[n].shape)
    except:
        print(n,sample[n])
        pass


In [ ]:
sample['cloud'].shape

In [ ]:
import matplotlib.pyplot as plt

def plot_contour(img, points,  cam_cx=0, cam_cy=0, cam_fx=0, cam_fy=0, trans=[[0, 0, 0]], rot_mat=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], store=False, jupyter=False, thickness =2, color=(0, 255, 0) ):
        """
        tag := tensorboard tag 
        epoch := tensorboard epoche
        store := ture -> stores the image to standard path
        path := != None creats the path and store to it path/tag.png
        img:= original_image, [widht,height,RGB], torch
        points:= points of the object model [length,x,y,z]
        trans: [1,3]
        rot: [3,3]
        """
        rot_mat = np.array(rot_mat)
        trans = np.array(trans)
        img_f = copy.deepcopy(img).numpy().astype(np.uint8)
        points = np.dot(points, rot_mat.T)
        points = np.add(points, trans[0, :])
        h = img_f.shape[0]
        w = img_f.shape[1]
        acc_array = np.zeros( (h,w,1),dtype = np.uint8)
        
        # project pointcloud onto image
        for i in range(0, points.shape[0]):
            p_x = points[i, 0]
            p_y = points[i, 1]
            p_z = points[i, 2]
            u = int(((p_x / p_z) * cam_fx) + cam_cx)
            v = int(((p_y / p_z) * cam_fy) + cam_cy)
            try:
                a = 10
                acc_array[v-a:v+a+1,u-a:u+a+1, 0] = 1
            except:
                pass
            
        kernel = np.ones((a*2,a*2,1),np.uint8)
        erosion = cv2.erode(acc_array ,kernel,iterations = 1)
        image, contours, hierarchy = cv2.findContours(np.expand_dims(erosion,2),cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        out = np.zeros( (h,w,3),dtype = np.uint8)
        cv2.drawContours(out, contours, -1, (0,255,0), 3)
        
        #display(Image.fromarray(out))
        
        for i in range(h):
            for j in range(w):
                if out[i,j,1] == 255: 
                    img_f[i,j,:] = out[i,j,:]
        display(Image.fromarray(img_f))
        
       
plot_contour( sample['img'], 
             sample['target'],
             sample['cam'][0],
             sample['cam'][1],
             sample['cam'][2],
             sample['cam'][3]) 
        
        

In [ ]:
import matplotlib.pyplot as plt
def check_if_edge(h,w,arr):
    try:
        a = np.sum(arr[:h-1,:w-1]) 
        b = np.sum(arr[:h-1,w+1:]) 
        c = np.sum(arr[h+1:,w+1:])
        d = np.sum(arr[h+1:,:w-1])
        #print(a,b,c,d)
        return a == 0 or b == 0  or c == 0 or d == 0
    except:
        return True
def plt_img(img):
    fig = plt.figure()
    fig.add_subplot(1, 1, 1)
    plt.imshow(img)
    plt.axis("off")
    plt.show()
def plt_img_grey(img):
    fig = plt.figure()
    fig.add_subplot(1, 1, 1)
    plt.imshow(img.squeeze(2))
    plt.axis("off")
    plt.show()
    
import scipy
def plot_contour(img, points,  cam_cx=0, cam_cy=0, cam_fx=0, cam_fy=0, trans=[[0, 0, 0]], rot_mat=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], store=False, jupyter=False, thickness =2, color=(0, 255, 0) ):
        """
        tag := tensorboard tag 
        epoch := tensorboard epoche
        store := ture -> stores the image to standard path
        path := != None creats the path and store to it path/tag.png
        img:= original_image, [widht,height,RGB]
        points:= points of the object model [length,x,y,z]
        trans: [1,3]
        rot: [3,3]
        """
        rot_mat = np.array(rot_mat)
        trans = np.array(trans)
        img_d = copy.deepcopy(img).numpy().astype(np.uint8)
        img_f = copy.deepcopy(img).numpy().astype(np.uint8)
        points = np.dot(points, rot_mat.T)
        points = np.add(points, trans[0, :])
        h = img_d.shape[0]
        w = img_d.shape[1]
        acc_array = np.zeros( (h,w,1),dtype = np.uint8)
        
        # project pointcloud onto image
        for i in range(0, points.shape[0]):
            p_x = points[i, 0]
            p_y = points[i, 1]
            p_z = points[i, 2]
            u = int(((p_x / p_z) * cam_fx) + cam_cx)
            v = int(((p_y / p_z) * cam_fy) + cam_cy)
            try:
                a = 10
                acc_array[v-a:v+a+1,u-a:u+a+1, 0] = 1
            except:
                pass
        # remove all points that are not on contour
        print("points added", np.sum(acc_array) )
        #plt_img( acc_array ) 
        for i in range(h):
            for j in range(w):
                if acc_array[i,j, 0] == 1:
                    if not check_if_edge(i,j,acc_array):
                        pass
                    else:
                        img_d[i,j,0] = 0
                        img_d[i,j,1] = 255
                        img_d[i,j,2] = 0
                        
        

        plt_img_grey( acc_array )  
        kernel = np.ones((a*2,a*2,1),np.uint8)
        erosion = cv2.erode(acc_array ,kernel,iterations = 1)

        image, contours, hierarchy = cv2.findContours(np.expand_dims(erosion,2),cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        #plt_img_grey(image)
        out = np.zeros( (h,w,3),dtype = np.uint8)
        #for c in contours:
        cv2.drawContours(out, contours, -1, (0,255,0), 3)
        #print(contours)
        #cv2.line(img_d, s, e, color, thickness)
        #cv2.line(image, start_point, end_point, color, thickness) 
        #print(image, contours)
        #display(Image.fromarray(image))
        display(Image.fromarray(out))
        #display(Image.fromarray(img))
        
        #plt_img_grey(image)
        #plt_img( out )     
        #plt_img(img_d)
        #plt_img(img)
        
        for i in range(h):
            for j in range(w):
                if out[i,j,1] == 255: 
                    img_f[i,j,:] = out[i,j,:]
        #plt_img(img_f)
        print(type(img_f),img_f.shape)
        display(Image.fromarray(img_f))
        # connect the points with lines.

plot_contour( sample['img'], 
             sample['target'],
             sample['cam'][0],
             sample['cam'][1],
             sample['cam'][2],
             sample['cam'][3]) 
        
        

In [ ]:

def plot_estimated_pose(img, points, trans=[[0, 0, 0]], rot_mat=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], cam_cx=0, cam_cy=0, cam_fx=0, cam_fy=0, store=False, jupyter=False, w=2):
        """
        tag := tensorboard tag 
        epoch := tensorboard epoche
        store := ture -> stores the image to standard path
        path := != None creats the path and store to it path/tag.png
        img:= original_image, [widht,height,RGB]
        points:= points of the object model [length,x,y,z]
        trans: [1,3]
        rot: [3,3]
        """

        img_d = copy.deepcopy(img)
        points = np.dot(points, rot_mat.T)
        points = np.add(points, trans[0, :])
        for i in range(0, points.shape[0]):
            p_x = points[i, 0]
            p_y = points[i, 1]
            p_z = points[i, 2]
            u = int(((p_x / p_z) * cam_fx) + cam_cx)
            v = int(((p_y / p_z) * cam_fy) + cam_cy)
            try:
                img_d[v - w:v + w + 1, u - w:u + w + 1, 0] = 0
                img_d[v - w:v + w + 1, u - w:u + w + 1, 1] = 255
                img_d[v - w:v + w + 1, u - w:u + w + 1, 0] = 0
            except:
                #print("out of bounce")
                pass

        if jupyter:
            display(Image.fromarray(img_d))

        if store:
            #store_ar = (img_d* 255).round().astype(np.uint8)
            #print("IMAGE D:" ,img_d,img_d.shape )
            save_image(img_d, tag=str(epoch) + tag, p_store=self.p_visu)
        if self.writer is not None:
            self.writer.add_image(tag, img_d.astype(
                np.uint8), global_step=epoch, dataformats='HWC')
            